In [ ]:
import numpy as np
import time
from pyrcn.datasets import load_digits
from sklearn.model_selection import train_test_split

from torchvision import transforms
from torch.utils.data import Dataset,DataLoader 

import torch
import matplotlib.pyplot as plt

In [ ]:
# 数据集加载类
class CustomDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        if transform is None:
              self.transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize(mean = (0.5,), std = (0.5,)),
                            ])
        else:
            self.transform = transform
        for i in range(len(X)):
            X[i] = np.flip(np.rot90(X[i],k=-1),1)
            X[i] = self.transform(X[i])
            Y[i] = Y[i][0]
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.Y)
    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        return x, y

In [ ]:
# 数据集加载并分割
X, y = load_digits(return_X_y=True, as_sequence=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
# 获得加载器
train_iteruse = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
test_iteruse = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [ ]:
# 前馈神经网络（人工神经网络），分为0个隐藏层（SLP）、1个隐藏层（MLP）和2个隐藏层（MLP）
class Flatten(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x.view(x.shape[0], -1)
class SoftmaxModel(torch.nn.Module):
    def __init__(self,hidden_layer_num):
        super().__init__()
        
        self.hidden_layer_num = hidden_layer_num
        
        self.flatten=Flatten()
        self.linear = None
        self.softmax=torch.nn.Softmax(dim=1)
        
        if self.hidden_layer_num == 0:
            self.linear = torch.nn.Sequential(torch.nn.Linear(64,10),torch.nn.ReLU())
        elif self.hidden_layer_num == 1:
            self.linear = torch.nn.Sequential(torch.nn.Linear(64,32),
                                              torch.nn.ReLU(),
                                              torch.nn.Linear(32,10),
                                              torch.nn.ReLU())
        elif self.hidden_layer_num == 2:
            self.linear = torch.nn.Sequential(torch.nn.Linear(64,32),
                                              torch.nn.ReLU(),
                                              torch.nn.Linear(32,16),
                                              torch.nn.ReLU(),
                                              torch.nn.Linear(16,10),
                                              torch.nn.ReLU())
        else:
            self.linear = None
    def forward(self,x):
        if self.linear is not None:
            x = self.flatten(x)
            x = x.to(torch.float32)
            x = self.linear(x)
            y_pred = self.softmax(x)
            return y_pred
        else:
            return None

In [ ]:
## 单层感知机 ##

#训练1000代
epoch  = 1000
model = SoftmaxModel(0)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.2)
train_loss_result=[]
train_acc_result=[]
test_loss_result=[]
test_acc_result=[]

total_time = 0 
for i in range(epoch):
    train_count_acc = 0
    train_loss = 0
    train_count= 0
    time_start = time.time()
    for batch,(x_softmax,y) in enumerate(train_iteruse):
        y = y.to(torch.long)
        y_predict = model.forward(x_softmax)
        loss_sum = criterion(y_predict, y).sum()
        optimizer.zero_grad()
        loss_sum.backward()
        optimizer.step()
        train_count_acc += (y_predict.argmax(dim=1) == y).sum().item()
        train_loss += loss_sum.item()
        train_count += y.shape[0]
    
    # print('epoch %d,loss %.6f, train acc %.3f'% (i + 1, train_loss/batch ,train_count_acc/train_count))
    train_loss_result.append(train_loss/batch)
    train_acc_result.append(train_count_acc/train_count)
    test_count_acc = 0
    test_loss = 0
    test_count= 0
    for batch,(x_softmax,y) in enumerate(test_iteruse):
        y = y.to(torch.long)
        y_predict = model.forward(x_softmax)
        test_count_acc += (y_predict.argmax(dim=1) == y).sum().item()
        loss_sum = criterion(y_predict, y).sum()
        test_loss +=loss_sum.item()
        test_count += y.shape[0]
    total_time += (time.time()-time_start)
    test_loss_result.append(test_loss/batch)
    test_acc_result.append(test_count_acc/test_count)
plt.subplot(2, 2, 1)
plt.plot(range(epoch), train_loss_result, color='green', label='training loss')

plt.xlabel('epoch')
plt.ylabel('training loss')
plt.subplot(2, 2, 2)
plt.plot(range(epoch), train_acc_result, color='red', label='training accuracy')
plt.xlabel('epoch')
plt.ylabel('training accuracy')
plt.subplot(2, 2, 3)
plt.plot(range(epoch), test_loss_result,  color='skyblue', label='testing loss')
plt.xlabel('epoch')
plt.ylabel('testing loss')
plt.subplot(2, 2, 4)
plt.plot(range(epoch), test_acc_result, color='blue', label='testing accuracy')
plt.xlabel('epoch')
plt.ylabel('testing accuracy')
plt.tight_layout()
plt.show()
print("模型训练总用时:"+str(total_time)+"秒")
print('train_loss_result is %.6f, train_acc_result is %.3f'% (train_loss_result[-1] ,train_acc_result[-1]))
print('test_loss_result is %.6f, test_acc_result is %.3f'% (test_loss_result[-1] ,test_acc_result[-1]))

In [ ]:
## 多层感知机（一层隐藏层） ##
#训练500代
epoch = 500
model = SoftmaxModel(1)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.2)
train_loss_result=[]
train_acc_result=[]
test_loss_result=[]
test_acc_result=[]

total_time = 0 
for i in range(epoch):
    train_count_acc = 0
    train_loss = 0
    train_count= 0
    time_start = time.time()
    for batch,(x_softmax,y) in enumerate(train_iteruse):
        y = y.to(torch.long)
        y_predict = model.forward(x_softmax)
        loss_sum = criterion(y_predict, y).sum()
        optimizer.zero_grad()
        loss_sum.backward()
        optimizer.step()
        train_count_acc += (y_predict.argmax(dim=1) == y).sum().item()
        train_loss += loss_sum.item()
        train_count += y.shape[0]
    
    # print('epoch %d,loss %.6f, train acc %.3f'% (i + 1, train_loss/batch ,train_count_acc/train_count))
    train_loss_result.append(train_loss/batch)
    train_acc_result.append(train_count_acc/train_count)
    test_count_acc = 0
    test_loss = 0
    test_count= 0
    for batch,(x_softmax,y) in enumerate(test_iteruse):
        y = y.to(torch.long)
        y_predict = model.forward(x_softmax)
        test_count_acc += (y_predict.argmax(dim=1) == y).sum().item()
        loss_sum = criterion(y_predict, y).sum()
        test_loss +=loss_sum.item()
        test_count += y.shape[0]
    total_time += (time.time()-time_start)
    test_loss_result.append(test_loss/batch)
    test_acc_result.append(test_count_acc/test_count)
plt.subplot(2, 2, 1)
plt.plot(range(epoch), train_loss_result, color='green', label='training loss')

plt.xlabel('epoch')
plt.ylabel('training loss')
plt.subplot(2, 2, 2)
plt.plot(range(epoch), train_acc_result, color='red', label='training accuracy')
plt.xlabel('epoch')
plt.ylabel('training accuracy')
plt.subplot(2, 2, 3)
plt.plot(range(epoch), test_loss_result,  color='skyblue', label='testing loss')
plt.xlabel('epoch')
plt.ylabel('testing loss')
plt.subplot(2, 2, 4)
plt.plot(range(epoch), test_acc_result, color='blue', label='testing accuracy')
plt.xlabel('epoch')
plt.ylabel('testing accuracy')
plt.tight_layout()
plt.show()
print("模型训练总用时:"+str(total_time)+"秒")
print('train_loss_result is %.6f, train_acc_result is %.3f'% (train_loss_result[-1] ,train_acc_result[-1]))
print('test_loss_result is %.6f, test_acc_result is %.3f'% (test_loss_result[-1] ,test_acc_result[-1]))

In [ ]:
## 多层感知机（二层隐藏层） ##
#训练500代
epoch  = 500
model = SoftmaxModel(2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.2)
train_loss_result=[]
train_acc_result=[]
test_loss_result=[]
test_acc_result=[]

total_time = 0 
for i in range(epoch):
    train_count_acc = 0
    train_loss = 0
    train_count= 0
    time_start = time.time()
    for batch,(x_softmax,y) in enumerate(train_iteruse):
        y = y.to(torch.long)
        y_predict = model.forward(x_softmax)
        loss_sum = criterion(y_predict, y).sum()
        optimizer.zero_grad()
        loss_sum.backward()
        optimizer.step()
        train_count_acc += (y_predict.argmax(dim=1) == y).sum().item()
        train_loss += loss_sum.item()
        train_count += y.shape[0]
    
    # print('epoch %d,loss %.6f, train acc %.3f'% (i + 1, train_loss/batch ,train_count_acc/train_count))
    train_loss_result.append(train_loss/batch)
    train_acc_result.append(train_count_acc/train_count)
    test_count_acc = 0
    test_loss = 0
    test_count= 0
    for batch,(x_softmax,y) in enumerate(test_iteruse):
        y = y.to(torch.long)
        y_predict = model.forward(x_softmax)
        test_count_acc += (y_predict.argmax(dim=1) == y).sum().item()
        loss_sum = criterion(y_predict, y).sum()
        test_loss +=loss_sum.item()
        test_count += y.shape[0]
    total_time += (time.time()-time_start)
    test_loss_result.append(test_loss/batch)
    test_acc_result.append(test_count_acc/test_count)
plt.subplot(2, 2, 1)
plt.plot(range(epoch), train_loss_result, color='green', label='training loss')

plt.xlabel('epoch')
plt.ylabel('training loss')
plt.subplot(2, 2, 2)
plt.plot(range(epoch), train_acc_result, color='red', label='training accuracy')
plt.xlabel('epoch')
plt.ylabel('training accuracy')
plt.subplot(2, 2, 3)
plt.plot(range(epoch), test_loss_result,  color='skyblue', label='testing loss')
plt.xlabel('epoch')
plt.ylabel('testing loss')
plt.subplot(2, 2, 4)
plt.plot(range(epoch), test_acc_result, color='blue', label='testing accuracy')
plt.xlabel('epoch')
plt.ylabel('testing accuracy')
plt.tight_layout()
plt.show()
print("模型训练总用时:"+str(total_time)+"秒")
print('train_loss_result is %.6f, train_acc_result is %.3f'% (train_loss_result[-1] ,train_acc_result[-1]))
print('test_loss_result is %.6f, test_acc_result is %.3f'% (test_loss_result[-1] ,test_acc_result[-1]))